In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
from PIL import Image
import os
import numpy as np
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((255,255)),
    torchvision.transforms.CenterCrop((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
model = models.vgg16(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 72.6MB/s]


In [6]:
with open("/content/sample_data/imagenet_class_index.txt","r") as f:
  lines = f.readlines()

class_idx_dict = {}
for line in lines:
  line = line.strip()
  key , value = line.split(':')
  key = key.strip()
  value = value.strip()
  class_idx_dict[key] = value


class_idx = [class_idx_dict[f"{i}"] for i in class_idx_dict.keys()]

In [7]:
for params in model.parameters():
  params.requires_grad == False

In [8]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# for name, layer in model.named_modules():
#   if isinstance(layer, torch.nn.Conv2d):
#     weights = layer.weight.shape
#     bias = layer.bias.shape
#     name = name.replace('features.','')
#     print(f"Conv Layer {name}")
#     print(f"Weights Shape : {weights}")
#     print(f"Bias Shape : ", bias if bias is not None else None )
#     print('-'*40)

In [10]:
name_list = {}
i=1
for name, layer in model.features.named_children():
  if isinstance(layer, torch.nn.Conv2d):
    name_list[i] = int(name)
    i += 1


In [11]:
print(name_list)

{1: 0, 2: 2, 3: 5, 4: 7, 5: 10, 6: 12, 7: 14, 8: 17, 9: 19, 10: 21, 11: 24, 12: 26, 13: 28}


In [12]:
user_input = int(input("Enter the Conv Layer which you need to see : "))
conv_layer_num = name_list[user_input]
conv_layer_name = "Conv_Layer" + str(conv_layer_num)

Enter the Conv Layer which you need to see : 13


In [ ]:
#filters

for name, layer in model.features.named_children():
  if int(name) == conv_layer_num:
    filter, bias = layer.weight, layer.bias

f_max, f_min = filter.max(), filter.min()
filter = (filter - f_min) / (f_max - f_min)

In [ ]:
# torch.Size([64, 3, 3, 3]) it is like (out_channels, input_channels, height, width)
# Filters
plt.figure(figsize=(5,5))
num_filter = filter.data.size(0)
print(num_filter)
print(filter.data[1].shape)
# print(filter.data.size)

for i in range(6):
  filter_data = filter.data[i].cpu().numpy().transpose(1,2,0)
  # print(filter_data.shape)
  num_channels = len(filter_data[0])
  for j in range(num_channels):
      plt.subplot(1, num_channels, j + 1)
      plt.imshow(filter_data[:,:,j], cmap='gray')  # Visualize each channel
      plt.axis('off')


#
print(filter.shape)
filter_data = filter.data.cpu().numpy().transpose(0,2,3,1)
print(filter.shape)
for j in range(1):
   plt.subplot(1, num_channels, j + 1)
   plt.imshow(filter_data[1,:,:,0], cmap='gray')  # Visualize each channel
   plt.axis('off')

In [13]:
# Feature Maps

conv_layers = {}

def get_conv_layers(name):
    def hook(module, input, output):
        conv_layers[name] = output.detach()
    return hook

# Register a forward hook to capture the output of the desired layer (conv1)
model.features[conv_layer_num].register_forward_hook(get_conv_layers(conv_layer_name))

image = Image.open('/content/sample_data/french_bulldog.jpeg')
transformed_image = transform(image)
final_image = transformed_image.unsqueeze(0)
final_image = final_image.to(device)
output = model(final_image)

final_output = conv_layers[conv_layer_name].squeeze(0)

final_data = final_output.cpu().numpy()
final_data = final_data.transpose(1,2,0)
s = final_data.shape

In [ ]:
fig = plt.figure(figsize=(30,20))
total_filters = s[2]
no_rows = (total_filters // 6) + 1
# total_grids =  no_rows * 6
# non_required_grids = total_grids - total_filters

grid = ImageGrid(fig, 111,
                 nrows_ncols=(no_rows, 6),
                 axes_pad=0.3,
                 )
for i in range(total_filters):
  grid[i].imshow(final_data[:,:,i-1], cmap='gray')
  grid[i].axis('off')


# for _ in range(non_required_grids):
#   grid[-1].remove()


plt.show()